#### Sme v Jupyter(lab) notebooku, ktorý mnohí poznáme a máme radi. 
#### Tí, čo ho nepoznajú, hádam ho s nami spoznajú z tej lepšej stránky. :-)
#### Notebook sa skladá z buniek, ako je táto, v ktorej práve sme (modrý pasik vľavo).
#### Bunky môžu byť typu `Markdown` (text, pokec) a `Code`, bunky s vykonávateľným Python kódom.
<hr/>

#### Treba nám dáta. Ak ich máme, môžeme rozmýšľať, ako ich čo najnázornejšie interpretovať.
#### Dnes nás budú sprevádzať __údaje o taxikároch v New Yorku__. Nie všetky, čo máme (vyše 11 mil. jázd).
#### Použijeme vzorku 310000 záznamov. Na prácu s dátami použijeme modul `polars` (rýchle `pandas`).
#### Importujeme ho. Bunku (typu `code`) vykonáme cez `Shift-Enter`, tým sa aj posunieme do ďalšej bunky.

In [ ]:
# komentar - preco nie pandas (rychlost, jednoduchost, perspektivnost)
import pandas as pd

In [ ]:
# nacitanie dat zo suboru, skusme si doplnanie cez TAB a pozrieme, kolko je moznosti nacitania
df = pd.read_parquet('data/nyc_taxi_sample.parq')

In [ ]:
df.describe()

In [ ]:
# co je df - DataFrame, tabulka s pomenovanymi stlpcami. ake su nase?
print(df.columns)   # co je co, uhadneme?

In [ ]:
# pozrieme datam na hlavu, alebo chvost, nie je explicitny index, iba cisla riadkov ako v zozname, list
df.head(8) # .tail()  # funkcii podhodit aka velka ma byt

In [ ]:
# len vybrane stlpce
df[['pick_dt', 'drop_dt']].head()  # zacina retazenie. zmenila sa df?
datumy = df[['pick_dt', 'drop_dt']]
datumy.head()

In [ ]:
# filtrovanie dat - co takto dostaneme?
from datetime import timedelta
# datumy[datumy['drop_dt'] < datumy['pick_dt']]
# datumy[datumy['drop_dt'] - datumy['pick_dt'] < timedelta(seconds=1)].shape
datumy[datumy['drop_dt'] - datumy['pick_dt'] > timedelta(hours=1)].shape
# datumy.filter(pl.col('drop_dt') - pl.col('pick_dt') > pl.duration(hours=1)).sort(by='pick_dt')
# datumy.filter(pl.col('drop_dt').dt.day() > pl.col('pick_dt').dt.day()).head() 

In [ ]:
def to_minutes(td):
    return td.days * 60 * 24 + td.seconds / 60

In [ ]:
df['rtime'] = (datumy['drop_dt'] - datumy['pick_dt']).apply(to_minutes).astype('float32')

In [ ]:
df['rtime'].head()

In [ ]:
# rozsah datumov
piky = df['pick_dt']  # rozdiel oproti datumy['pick_dt'] (ako v polars)
dropy = df['drop_dt']
# datumy['pick_dt']   # .columns
dropy.min(), dropy.max()

### Niektoré jazdy nedávajú zmysel. Porozmýšľame, vyčistíme.

In [ ]:
# velke casy jazd 1440/60 je 24 hod.!
# df_rtimes = 
df[df['rtime'] > 1400].shape # asi im osmyklo o den, dame prec

In [ ]:
df['rtime'].max()

In [ ]:
# co je tu podozrive?
df[df['passengers'] == 0] #  & (pl.col('fare') > 0)) # ako interpretovat?
# df_rtimes.filter((pl.col('fare') <= 0)) # .select(pl.col('distance')).max()
# df.filter((pl.col('tip') < 0))

In [ ]:
# ako rozhodneme?
# co s nulovymi vzdialenostami? zapornymi platbami?
# ...

In [ ]:
# ked 0 dat 1 pasaziera, hoci  by bol aj taxikar, neskor vycistime nezalatene
 
df['passengers'] = df['passengers'].where(df['passengers'] > 0, other=1)

In [ ]:
df['passengers'].min()

In [ ]:
df.shape

In [ ]:
# odstranit zaporne a nulove platby
df = df[df['fare'] > 0]

In [ ]:
# podozrivo velka max. vzdialenost, kde to bolo? neustriehli sme...
# df_rtimes.filter(pl.col('distance') == df_rtimes['distance'].max())
df = df[df['distance'] < 45] # hadam v NY nebudu vacsie vzdial.

In [ ]:
# velka platba, 500 $
# df_rtimes.filter(pl.col('fare') == df_rtimes['fare'].max())
# nechame pod 90 $ a tringel pod 40 $
df = df[df['fare'] < 90]
df = df[df['tip'] < 40]

In [ ]:
df = df[df['rtime'] < 120]

In [ ]:
# zapiseme a dalej budeme robit s touto framou
print(df.shape)
df.to_parquet('data/nyc_taxi310k.parq')

#### Zhrnutie, čo vieme
- `select, filter, with_columns, alias`
- pridali sme stĺpec `rtimes`, trvanie jázd